# Data Preparation

## Fetch Data from CSV

In [1]:
%pip install numpy pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd

In [3]:
question_path = './data/dataset-question.csv'
answer_path = './data/dataset-answer.csv'

question_attr = ['pattern', 'tag']
answer_attr = ['responses', 'tag']

question_data = pd.read_csv(question_path)[question_attr]
answer_data = pd.read_csv(answer_path)[answer_attr]

# data_answer.head(10)
print("Jumlah pertanyaan: ", question_data.shape)
print("Jumlah pertanyaan: ", answer_data.shape)

Jumlah pertanyaan:  (259, 2)
Jumlah pertanyaan:  (17, 2)


In [4]:
question_data.head(10)

,pattern,tag
0,Hi,gr_hi
1,Halo,gr_ha
2,Selamat Pagi,gr_pa
3,Selamat Siang,gr_si
4,Selamat Sore,gr_so
5,Selamat Malam,gr_ma
6,Ada program studi apa saja di UIN Sunan Gunung...,prodi
7,Apa saja jurusan di UIN Bandung,prodi
8,Ada jurusan apa saja?,prodi
9,Mohon informasi mengenai program studi yang te...,prodi


In [5]:
answer_data.head(10)

,responses,tag
0,Halo! Apa yang ingin anda cari tahu hari ini?,gr_hi
1,Hi! Apa yang bisa kami bantu?,gr_ha
2,Selamat Pagi! Apa yang bisa kami bantu untuk m...,gr_pa
3,Selamat Siang! Apa yang ingin anda cari tahu?,gr_si
4,Selamat Sore! Apa yang ingin anda ketahui?,gr_so
5,Selamat Malam! Apa yang bisa kami bantu?,gr_ma
6,UIN Sunan Gunung Djati Bandung menyelenggaraka...,prodi
7,SNBP adalah singkatan dari Seleksi Nasional Be...,jm_s1
8,Untuk jurusan yang bisa dipilih pada Seleksi M...,snbp_jur
9,Untuk jurusan yang bisa dipilih pada Seleksi M...,snbt_jur


## Data Cleaning

In [6]:
%pip install Sastrawi transformers tensorflow


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from transformers import AutoTokenizer
import re
import tensorflow as tf

c:\Projects\uni\python\skripsi\SBERT_chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
SBERT_TOKENIZER = 'firqaaa/indo-sentence-bert-base'

class Preprocess():
    def __init__(self, max_len=128):
        self.stemmer = StemmerFactory().create_stemmer()
        self.stopword = StopWordRemoverFactory().create_stop_word_remover()
        self.tokenizer = AutoTokenizer.from_pretrained(SBERT_TOKENIZER)
        self.max_len = max_len

    def cleaning(self, val):
        val = re.sub(r'\s+', ' ', val)
        val = re.sub("[^a-zA-Z0-9;]", " ", val)
        return val
    
    def casefolding(self, val):
        return str(val).lower()
    
    def stemming(self, val):
        return self.stemmer.stem(str(val))
    
    def stopwordRemoval(self, val):
        return self.stopword.remove(str(val))
    
    def embedding(self, val):
        return self.tokenizer.encode_plus(
            val,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='tf'
        )
    
    def preprocessing(self, sentences):
        for i in range(len(sentences)):
            input = sentences[i]
            input = self.cleaning(input)
            input = self.casefolding(input)
            input = self.stemming(input)
            input = self.stopwordremove(input)
            sentences[i] = input
        return sentences

    def tokenizing(self, sentences):
        input_ids, attention_mask = [], []
        for sentence in sentences:
            output = self.embedding(sentence)
            input_ids.append(output['input_ids'])
            attention_mask.append(output['attention_mask'])
        return {
            'input_ids': tf.convert_to_tensor(
                np.asarray(input_ids).squeeze(),
                dtype=tf.int32
            ),
            'attention_mask': tf.convert_to_tensor(
                np.asarray(attention_mask).squeeze(),
                dtype=tf.int32
            )
        }

    def preprocess_get_token(self, sentences, display_len=20):
        preprocessing = self.preprocessing(sentences)
        tokenized = self.tokenizing(preprocessing)
        return [self.tokenizer.convert_ids_to_tokens(tokenized['input_ids'][i][:display_len]) for i in range(len(sentences))]

preprocess = Preprocess()    

### Casefolding

In [9]:
sample_texts = question_data['pattern']
print(sample_texts[7])
print(sample_texts[8])
print(sample_texts[9])

Apa saja jurusan di UIN Bandung
Ada jurusan apa saja?
Mohon informasi mengenai program studi yang tersedia di UIN Sunan Gunung Djati Bandung.


In [10]:
preprocess = Preprocess()
hasil_casefolding = [preprocess.casefolding(sample) for sample in sample_texts]

bf_cf = sample_texts
af_cf = hasil_casefolding

df = pd.DataFrame({
    'Before Casefolding': bf_cf,
    'After Casefolding': af_cf 
})

df.tail(10)

,Before Casefolding,After Casefolding
249,Ada beasiswa di UIN Sunan Gunung Djati Bandung?,ada beasiswa di uin sunan gunung djati bandung?
250,Beasiswa ada di UIN Sunan Gunung Djati Bandung...,beasiswa ada di uin sunan gunung djati bandung...
251,Di UIN Sunan Gunung Djati Bandung ada beasiswa...,di uin sunan gunung djati bandung ada beasiswa...
252,Beasiswa di UIN Sunan Gunung Djati Bandung ada...,beasiswa di uin sunan gunung djati bandung ada...
253,UIN Sunan Gunung Djati Bandung punya beasiswa?,uin sunan gunung djati bandung punya beasiswa?
254,Ada info soal beasiswa di UIN Sunan Gunung Dja...,ada info soal beasiswa di uin sunan gunung dja...
255,Beasiswa di UIN Sunan Gunung Djati Bandung ada...,beasiswa di uin sunan gunung djati bandung ada...
256,"Minta info dong, di UIN Sunan Gunung Djati Ban...","minta info dong, di uin sunan gunung djati ban..."
257,Beasiswa di UIN Sunan Gunung Djati Bandung ada...,beasiswa di uin sunan gunung djati bandung ada...
258,UIN Sunan Gunung Djati Bandung ada beasiswa?,uin sunan gunung djati bandung ada beasiswa?


### Cleaning

In [11]:
preprocess = Preprocess()
hasil_cleaning = [preprocess.cleaning(sample) for sample in hasil_casefolding]
bf_cl = hasil_casefolding
af_cl = hasil_cleaning
df = pd.DataFrame({
    'Before Cleaning': bf_cl,
    'After Cleaning': af_cl
})
df.tail(10)

,Before Cleaning,After Cleaning
249,ada beasiswa di uin sunan gunung djati bandung?,ada beasiswa di uin sunan gunung djati bandung
250,beasiswa ada di uin sunan gunung djati bandung...,beasiswa ada di uin sunan gunung djati bandung...
251,di uin sunan gunung djati bandung ada beasiswa...,di uin sunan gunung djati bandung ada beasiswa...
252,beasiswa di uin sunan gunung djati bandung ada...,beasiswa di uin sunan gunung djati bandung ada...
253,uin sunan gunung djati bandung punya beasiswa?,uin sunan gunung djati bandung punya beasiswa
254,ada info soal beasiswa di uin sunan gunung dja...,ada info soal beasiswa di uin sunan gunung dja...
255,beasiswa di uin sunan gunung djati bandung ada...,beasiswa di uin sunan gunung djati bandung ada...
256,"minta info dong, di uin sunan gunung djati ban...",minta info dong di uin sunan gunung djati ban...
257,beasiswa di uin sunan gunung djati bandung ada...,beasiswa di uin sunan gunung djati bandung ada...
258,uin sunan gunung djati bandung ada beasiswa?,uin sunan gunung djati bandung ada beasiswa


### Stemming

In [13]:
preprocess = Preprocess()
hasil_stemming = [preprocess.stemming(sample) for sample in hasil_cleaning]
bf_st = hasil_cleaning
af_st = hasil_stemming
df = pd.DataFrame({
    "Before Stemming" : bf_st,
    "After Stemming" : af_st
})
df.tail(10)

,Before Stemming,After Stemming
249,ada beasiswa di uin sunan gunung djati bandung,ada beasiswa di uin sunan gunung djati bandung
250,beasiswa ada di uin sunan gunung djati bandung...,beasiswa ada di uin sunan gunung djati bandung...
251,di uin sunan gunung djati bandung ada beasiswa...,di uin sunan gunung djati bandung ada beasiswa...
252,beasiswa di uin sunan gunung djati bandung ada...,beasiswa di uin sunan gunung djati bandung ada...
253,uin sunan gunung djati bandung punya beasiswa,uin sunan gunung djati bandung punya beasiswa
254,ada info soal beasiswa di uin sunan gunung dja...,ada info soal beasiswa di uin sunan gunung dja...
255,beasiswa di uin sunan gunung djati bandung ada...,beasiswa di uin sunan gunung djati bandung ada...
256,minta info dong di uin sunan gunung djati ban...,minta info dong di uin sunan gunung djati band...
257,beasiswa di uin sunan gunung djati bandung ada...,beasiswa di uin sunan gunung djati bandung ada...
258,uin sunan gunung djati bandung ada beasiswa,uin sunan gunung djati bandung ada beasiswa


### Stopword Removal

In [14]:
preprocess = Preprocess()
hasil_stopword_removal = [preprocess.stopwordRemoval(sample) for sample in hasil_stemming]
bf_sr = hasil_stemming
af_sr = hasil_stopword_removal
df = pd.DataFrame({
    "Before Stopword Removal": bf_sr,
    "After Stopword Removal": af_sr
})
df.tail(10)

,Before Stopword Removal,After Stopword Removal
249,ada beasiswa di uin sunan gunung djati bandung,beasiswa uin sunan gunung djati bandung
250,beasiswa ada di uin sunan gunung djati bandung...,beasiswa di uin sunan gunung djati bandung
251,di uin sunan gunung djati bandung ada beasiswa...,uin sunan gunung djati bandung beasiswa
252,beasiswa di uin sunan gunung djati bandung ada...,beasiswa uin sunan gunung djati bandung nggak
253,uin sunan gunung djati bandung punya beasiswa,uin sunan gunung djati bandung punya beasiswa
254,ada info soal beasiswa di uin sunan gunung dja...,info soal beasiswa uin sunan gunung djati bandung
255,beasiswa di uin sunan gunung djati bandung ada...,beasiswa uin sunan gunung djati bandung info l...
256,minta info dong di uin sunan gunung djati band...,minta info dong uin sunan gunung djati bandung...
257,beasiswa di uin sunan gunung djati bandung ada...,beasiswa uin sunan gunung djati bandung gak sih
258,uin sunan gunung djati bandung ada beasiswa,uin sunan gunung djati bandung beasiswa


### Filtering (Drop Duplicate Data)

In [21]:
df = pd.DataFrame(hasil_stopword_removal, columns=['patterns'])
hasil_filtering = df.drop_duplicates(subset=['patterns'])
print('Filtered data', hasil_filtering.shape)
df.tail(10)

Filtered data (247, 1)


,patterns
249,beasiswa uin sunan gunung djati bandung
250,beasiswa di uin sunan gunung djati bandung
251,uin sunan gunung djati bandung beasiswa
252,beasiswa uin sunan gunung djati bandung nggak
253,uin sunan gunung djati bandung punya beasiswa
254,info soal beasiswa uin sunan gunung djati bandung
255,beasiswa uin sunan gunung djati bandung info l...
256,minta info dong uin sunan gunung djati bandung...
257,beasiswa uin sunan gunung djati bandung gak sih
258,uin sunan gunung djati bandung beasiswa


### Cleaning Result

In [22]:
preprocessed_data = hasil_filtering
print("total data: ", preprocessed_data.shape)

total data:  (247, 1)


## Labelling

In [23]:
# Defining Labels
LABELS = answer_data['tag'].tolist()
len(LABELS), LABELS

(17,
 ['gr_hi',
  'gr_ha',
  'gr_pa',
  'gr_si',
  'gr_so',
  'gr_ma',
  'prodi',
  'jm_s1',
  'snbp_jur',
  'snbt_jur',
  'spanpt_jur',
  'umpt_jur',
  'man_jur',
  'jm_s2',
  'portal_pmb',
  'ukt',
  'beasiswa'])

## Model

In [12]:
# from transformers import AutoTokenizer, AutoModel
# import torch


# #Mean Pooling - Take attention mask into account for correct averaging
# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# # Sentences we want sentence embeddings for
# sentences = ["Ibukota Perancis adalah Paris", 
#              "Menara Eifel terletak di Paris, Perancis", 
#              "Pizza adalah makanan khas Italia", 
#              "Saya kuliah di Carneige Mellon University"]


# # Load model from HuggingFace Hub
# tokenizer = AutoTokenizer.from_pretrained('firqaaa/indo-sentence-bert-base')
# model = AutoModel.from_pretrained('firqaaa/indo-sentence-bert-base')

# # Tokenize sentences
# encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# # Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)

# # Perform pooling. In this case, mean pooling.
# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# print("Sentence embeddings:")
# print(sentence_embeddings)